## Deploying two modules on IoT Edge Device

In this notebook, we deploy two modules -- image-capture-od and people-detection-service, on IoT Edge.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import time
from dotenv import set_key, get_key, find_dotenv

In [2]:
# get the .env file where all the variables are stored
env_path = find_dotenv(raise_error_if_not_found=True)

In [3]:
resource_group = get_key(env_path, 'resource_group')
iot_hub_name = get_key(env_path, 'iot_hub_name')
device_id = get_key(env_path, 'device_id')


acr_name = get_key(env_path, 'acr_name')
acr_login_server = get_key(env_path, 'acr_login_server')
acr_password = get_key(env_path, 'acr_password')
img1_location = get_key(env_path, 'img1_location')
img2_location = get_key(env_path, 'img2_location')

In [4]:
# deploy two modules
module1_name = 'image-capture-od'
module2_name = 'people-detection-service'

In [5]:
# Based on the deployment_template.json file, we insert necessary information.

file = open('./deployment_annotation_template.json')
contents = file.read()

contents = contents.replace('__REGISTRY_SERVER_NAME', acr_login_server)
contents = contents.replace('__REGISTRY_PASSWORD', acr_password)
contents = contents.replace('__REGISTRY_USER_NAME', acr_name)
contents = contents.replace('__MODULE1_NAME', module1_name)
contents = contents.replace('__MODULE2_NAME', module2_name)
contents = contents.replace('__REGISTRY_IMAGE1_LOCATION', img1_location)
contents = contents.replace('__REGISTRY_IMAGE2_LOCATION', img2_location)

with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [ ]:
# Push the deployment JSON to the IOT Hub
!az iot edge set-modules --device-id $device_id \
                         --hub-name $iot_hub_name \
                         --content deployment.json \
                         -g $resource_group

When you run `docker ps` command in the edge device, you should see there are four containers running: `edgeAgent`, `edgeHub`, and two other containers with name `module1_name` and `module2_name`.

In [7]:
# restart the daemon
!sudo systemctl restart iotedge
time.sleep(20) # Wait 20 seconds for iotedge to restart
# restart the daemon again
!sudo systemctl restart iotedge

In [8]:
!docker ps

CONTAINER ID        IMAGE                                      COMMAND                   CREATED             STATUS              PORTS                                                                  NAMES
bac4cdde977a        yanzacr123.azurecr.io/img2:v1              "/bin/sh -c 'python …"    53 seconds ago      Up 13 seconds       0.0.0.0:80->80/tcp                                                     people-detection-service
d1844d89d91e        yanzacr123.azurecr.io/img1:v1              "python3 -u ./main.py"    55 seconds ago      Up 14 seconds       0.0.0.0:5012->5012/tcp                                                 image-capture-od
2169246a84d9        mcr.microsoft.com/azureiotedge-hub:1.0     "/bin/sh -c 'echo \"$…"   13 hours ago        Up 7 seconds        0.0.0.0:443->443/tcp, 0.0.0.0:5671->5671/tcp, 0.0.0.0:8883->8883/tcp   edgeHub
3b55443f4921        mcr.microsoft.com/azureiotedge-agent:1.0   "/bin/sh -c 'echo \"$…"   14 hours ago        Up 28 seconds                      

### Verify Deployment Succesfull

In [ ]:
dbe_name = get_key(env_path, 'dbe_name')
dbe_ip_address = get_key(env_path, 'dbe_ip_address')

1. In Azure portal `Data Box Edge / Data Box Gatewa` resource (the resource name is `dbe_name`, which can be obtained by executing previous cell), please click `Modules`. It should show four container running.
2. Open a browser from a computer and type in address `http://<dbe_ip_address>:5012/`, you should see a video stream playing. Below image shows a sample frame of the stream vido. The `dbe_ip_address` is obtained by executing previous cell.

![stream_video_frame](video_stream_frame.png)


In [ ]:
Next we will proceed with notebook [04_DeployOnIOTedge.ipynb](./04_DeployOnIOTedge.ipynb).